In [ ]:
source('../src/requirements.R')
source('../src/utils.R')
#INLA:::inla.dynload.workaround() ## LINUX WORKAROUND

# Read files

In [ ]:
admunit = 'county'

source = 'mobility_google'
response_var = 'workplaces_percent_change_from_baseline_interp_ma_diff_maxmin'
phase = 'drop'
pp_checks <- data.frame(matrix(ncol=6,nrow=0, dimnames=list(NULL, c("model", "ags_var", "bayes_r2_train", "bayes_r2_test", "waic", "cpo_fail"))))
filename = paste0(source,'_',admunit,'_',phase)

In [ ]:
## Read geometries
data_sp <- readOGR(paste0("../data/",filename,".shp"))
data_sp_states <- readOGR(paste0("../data/us_states_geoms.shp"))

## Read data (55% PCs)
pc_thr = '0.55'
data_55 <- read.delim(paste0("../data/",filename,'_',pc_thr,'.csv'), 
                      header = TRUE, 
                      sep = ',',na.strings = "",  colClasses = list(geoid = "character"))
cov_55 = c(colnames(select(data_55,contains("pc_"))),'cases_cum_dens')


## Read data (90% PCs)
pc_thr = '0.9'
data <- read.delim(paste0("../data/",filename,'_',pc_thr,'.csv'), 
                      header = TRUE, 
                      sep = ',',na.strings = "",colClasses = list(geoid = "character"))
cov_90 = c(colnames(select(data,contains("pc_"))),'cases_cum_dens')

## Proportion of cases per 100,000 people
data['cases_cum_dens'] = data['cases_cum_dens']*100000

## Add state ID
data = merge(data,data_sp_states@data[,c('name','ID')],
             by.x='sub_region_1', by.y='name',how = 'left',
             suffixes = c("","_sub_region_1"))
data$ID_sub_region_1 <- as.numeric(data$ID_sub_region_1)

In [ ]:
dim(data)

# Compute adjecency matrix for US counties

In [ ]:
#Compute first order neighbours
data_nb <- poly2nb(data_sp, row.names = data_sp$ID)
names(data_nb) <- attr(data_nb, "region.id")

#Convert it to a binary adjacency matrix
data_adj <- nb2mat(data_nb , style = "B",zero.policy=TRUE)

# Scale covariates

In [ ]:
data[c(cov_90)] <- lapply(data[c(cov_90)], function(x) c(scale(x)))

# Priors

In [ ]:
prior.iid = c(1,0.001)
prior.besag = c(1,0.001)

# Held-out data

In [ ]:
data[['y']] <- data[[response_var]]
data[['y_scaled']] <- data[[response_var]]/100

In [ ]:
smp_size <- floor(0.7 * nrow(data))

set.seed(123)
train_ind <- sample(seq_len(nrow(data)), size = smp_size)

data_train <- data[train_ind, ]
data_test <- data[-train_ind, ]

data[-train_ind, 'y'] <- NA
data[-train_ind, 'y_scaled'] <- NA

# Models

## 0. Fixed effects only - log transform, Gaussian likelihood, 55% retained AGS variance

In [ ]:
ff  <- as.formula(paste('y_scaled', paste(c(cov_55),
                                       collapse=" + "),sep='~'))

out55_gau <- inla(ff,data = data, 
            control.compute = list(cpo=TRUE,waic = TRUE),
            control.predictor = list(compute = TRUE, link = 1), 
            control.fixed = list(mean = 0, prec = 1, prec.intercept = 1), 
            num.threads=1, 
            verbose = TRUE)

summary(out55_gau)

### Compute odds percentage relative change

In [ ]:
fixed = round(out55_gau$summary.fixed,6)
fixed$sig = ifelse(0> fixed['0.025quant'] & 0 < fixed['0.975quant'], 'non_sig','sig')
fixed$pc_increase = c(NA,(exp(fixed$mean[-1])-1)*100)
fixed

In [ ]:
for(i in seq(1,length(out55_gau$marginals.fixed))){
    m = out55_gau$marginals.fixed[[i]]
    print(paste0(round(inla.beta.marginal.summary(m)[[1]],2)," (",round(inla.beta.marginal.summary(m)[[2]],2),")"))
}

In [ ]:
write.table(fixed,paste0("../data/",filename,'_odds_fixed_55_gau.csv'), 
            col.names = TRUE, row.names = FALSE, sep=',')

### Predictive checks

In [ ]:
pp_checks[1,'model'] <- 'fixed'
pp_checks[1,'ags_var'] <- '55'
pp_checks[1,'lik'] <- 'gaussian'
pp_checks[1,'bayes_r2_train'] <- bayes_pseudoR2(out55_gau, data_train[['y_scaled']],index_obs = as.numeric(row.names(data_train)))
pp_checks[1,'bayes_r2_test'] <- bayes_pseudoR2(out55_gau, data_test[['y_scaled']],index_obs = as.numeric(row.names(data_test)))
pp_checks[1,'waic'] <- out55_gau$waic$waic
pp_checks[1,'cpo_fail'] <- sum(out55_gau$cpo$failure, na.rm = TRUE)/dim(data)[1]*100
pp_checks

In [ ]:
pp_checks_train <- inla_ppcheck(out55_gau, data_train[['y_scaled']], index_obs = as.numeric(row.names(data_train)))
pp_checks_test <- inla_ppcheck(out55_gau, data_test[['y_scaled']], index_obs = as.numeric(row.names(data_test)))
options(repr.plot.width = 40, repr.plot.height = 12.5)
p <- ggplot_inla_ppcheck(pp_checks_train, pp_checks_test,CI = TRUE, binwidth=0.05)
ggsave(paste0("../plots/",filename,"_fixed_55_gau_pp_checks.pdf"), p, width = 42, height = 10)

plot(p)

## 1. Fixed effects only - Beta likelihood, 55% retained AGS variance

In [ ]:
ff  <- as.formula(paste('y_scaled', paste(c(cov_55),
                                            collapse=" + "),sep='~'))

out55 <- inla(ff,data = data, 
            family = 'beta',
            control.compute = list(cpo=TRUE,waic = TRUE),
            control.predictor = list(compute = TRUE, link = 1), 
            control.fixed = list(mean = 0, prec = 1, prec.intercept = 1), 
            num.threads=1, 
            verbose = TRUE)

summary(out55)

### Compute odds percentage relative change

In [ ]:
fixed = round(out55$summary.fixed,6)
fixed$sig = ifelse(0> fixed['0.025quant'] & 0 < fixed['0.975quant'], 'non_sig','sig')
fixed$pc_increase = c(NA,(exp(fixed$mean[-1])-1)*100)
fixed

In [ ]:
for(i in seq(1,length(out55$marginals.fixed))){
    m = out55$marginals.fixed[[i]]
    print(paste0(round(inla.beta.marginal.summary(m)[[1]],2)," (",round(inla.beta.marginal.summary(m)[[2]],2),")"))
}

In [ ]:
write.table(fixed,paste0("../data/",filename,'_odds_fixed_55.csv'), 
            col.names = TRUE, row.names = FALSE, sep=',')

### Predictive checks

In [ ]:
pp_checks[2,'model'] <- 'fixed'
pp_checks[2,'ags_var'] <- '55'
pp_checks[2,'lik'] <- 'beta'
pp_checks[2,'bayes_r2_train'] <- bayes_pseudoR2(out55, data_train[['y_scaled']],index_obs = as.numeric(row.names(data_train)))
pp_checks[2,'bayes_r2_test'] <- bayes_pseudoR2(out55, data_test[['y_scaled']],index_obs = as.numeric(row.names(data_test)))
pp_checks[2,'waic'] <- out55$waic$waic
pp_checks[2,'cpo_fail'] <- sum(out55$cpo$failure, na.rm = TRUE)/dim(data)[1]*100
pp_checks

In [ ]:
pp_checks_train <- inla_ppcheck(out55, data_train[['y_scaled']], index_obs = as.numeric(row.names(data_train)))
pp_checks_test <- inla_ppcheck(out55, data_test[['y_scaled']], index_obs = as.numeric(row.names(data_test)))
options(repr.plot.width = 40, repr.plot.height = 12.5)
p <- ggplot_inla_ppcheck(pp_checks_train, pp_checks_test,CI = TRUE, binwidth=0.05)
ggsave(paste0("../plots/",filename,"_fixed_55_pp_checks.pdf"), p, width = 42, height = 10)

plot(p)

## 2. Fixed effects only - Beta likelihood, 90% retained AGS variance

In [ ]:
ff  <- as.formula(paste('y_scaled', paste(c(cov_90),
                                            collapse=" + "),sep='~'))

out90 <- inla(ff,data = data, 
            family = 'beta',
            control.compute = list(cpo=TRUE,waic = TRUE),
            control.predictor = list(compute = TRUE, link = 1), 
            control.fixed = list(mean = 0, prec = 1, prec.intercept = 1), 
            num.threads=1, 
            verbose = TRUE)

summary(out90)

### Compute odds percentage relative change

In [ ]:
fixed = round(out90$summary.fixed,6)
fixed$sig = ifelse(0> fixed['0.025quant'] & 0 < fixed['0.975quant'], 'non_sig','sig')
fixed$pc_increase = c(NA,(exp(fixed$mean[-1])-1)*100)
fixed

In [ ]:
for(i in seq(1,length(out90$marginals.fixed))){
    m = out90$marginals.fixed[[i]]
    print(paste0(round(inla.beta.marginal.summary(m)[[1]],2)," (",round(inla.beta.marginal.summary(m)[[2]],2),")"))
}

In [ ]:
write.table(fixed,paste0("../data/",filename,'_odds_fixed_90.csv'), 
            col.names = TRUE, row.names = FALSE, sep=',')

### Predictive checks

In [ ]:
pp_checks[3,'model'] <- 'fixed'
pp_checks[3,'ags_var'] <- '90'
pp_checks[3,'lik'] <- 'beta'
pp_checks[3,'bayes_r2_train'] <- bayes_pseudoR2(out90, data_train[['y_scaled']],index_obs = as.numeric(row.names(data_train)))
pp_checks[3,'bayes_r2_test'] <- bayes_pseudoR2(out90, data_test[['y_scaled']],index_obs = as.numeric(row.names(data_test)))
pp_checks[3,'waic'] <- out90$waic$waic
pp_checks[3,'cpo_fail'] <- sum(out90$cpo$failure, na.rm = TRUE)/dim(data)[1]*100
pp_checks

In [ ]:
pp_checks_train <- inla_ppcheck(out90, data_train[['y_scaled']], index_obs = as.numeric(row.names(data_train)))
pp_checks_test <- inla_ppcheck(out90, data_test[['y_scaled']], index_obs = as.numeric(row.names(data_test)))
options(repr.plot.width = 40, repr.plot.height = 12.5)
p <- ggplot_inla_ppcheck(pp_checks_train, pp_checks_test,CI = TRUE, binwidth=0.05)
ggsave(paste0("../plots/",filename,"_90_fixed_pp_checks.pdf"), p, width = 42, height = 10)

plot(p)

## 2. IID only - Beta likelihood, 55% retained AGS variance

In [ ]:
ff.iid  <- as.formula(paste('y_scaled', paste(c("f(ID, param = c(prior.iid), model = 'iid')",
                                              cov_55),
                                              collapse=" + "),sep='~'))

out.iid <- inla(ff.iid,data = data,
                family = 'beta',
                control.compute = list(cpo=TRUE,waic = TRUE),
                control.predictor = list(compute = TRUE, link = 1), 
                control.fixed = list(mean = 0, prec = 1, prec.intercept=1), 
                num.threads=1, 
                verbose = TRUE)

summary(out.iid)

### Compute odds percentage relative change

In [ ]:
fixed = round(out.iid$summary.fixed,6)
fixed$sig = ifelse(0> fixed['0.025quant'] & 0 < fixed['0.975quant'], 'non_sig','sig')
fixed$pc_increase = c(NA,(exp(fixed$mean[-1])-1)*100)
fixed

In [ ]:
for(i in seq(1,length(out.iid$marginals.fixed))){
    m = out.iid$marginals.fixed[[i]]
    print(paste0(round(inla.beta.marginal.summary(m)[[1]],2)," (",round(inla.beta.marginal.summary(m)[[2]],2),")"))
}

In [ ]:
write.table(fixed,paste0("../data/",filename,'_odds_iid.csv'), 
            col.names = TRUE, row.names = FALSE, sep=',')

### Predictive checks

In [ ]:
pp_checks[4,'model'] <- 'iid'
pp_checks[4,'ags_var'] <- '55'
pp_checks[4,'lik'] <- 'beta'
pp_checks[4,'bayes_r2_train'] <- bayes_pseudoR2(out.iid, data_train[['y_scaled']],index_obs = as.numeric(row.names(data_train)))
pp_checks[4,'bayes_r2_test'] <- bayes_pseudoR2(out.iid, data_test[['y_scaled']],index_obs = as.numeric(row.names(data_test)))
pp_checks[4,'waic'] <- out.iid$waic$waic
pp_checks[4,'cpo_fail'] <- sum(out.iid$cpo$failure, na.rm = TRUE)/dim(data)[1]*100
pp_checks

In [ ]:
pp_checks_train <- inla_ppcheck(out.iid, data_train[['y_scaled']], index_obs = as.numeric(row.names(data_train)))
pp_checks_test <- inla_ppcheck(out.iid, data_test[['y_scaled']], index_obs = as.numeric(row.names(data_test)))
options(repr.plot.width = 40, repr.plot.height = 12.5)
p <- ggplot_inla_ppcheck(pp_checks_train, pp_checks_test,CI = TRUE, binwidth=0.05)
ggsave(paste0("../plots/",filename,"_iid_pp_checks.pdf"), p, width = 42, height = 10)

plot(p)

## 3. IID + Spatial random effect model - Beta likelihood, 55% retained AGS variance

In [ ]:
ff.sre <- as.formula(paste('y_scaled', paste(c("f(ID, model = 'bym', param = c(prior.iid,prior.besag), graph=data_adj, adjust.for.con.comp = TRUE, scale.model = TRUE)",
                                             cov_55),
                                             collapse=" + "),sep='~'))

out.sre <- inla(ff.sre,data = data, 
                family = 'beta',
                control.compute = list(cpo=TRUE,waic = TRUE),
                control.predictor = list(compute = TRUE, link = 1), 
                control.fixed = list(mean = 0, prec = 1, prec.intercept=1),
                num.threads=1,
                verbose = TRUE)
out.sre = inla.rerun(out.sre)
summary(out.sre)

### Compute odds percentage relative change

In [ ]:
fixed = round(out.sre$summary.fixed,6)
fixed$sig = ifelse(0> fixed['0.025quant'] & 0 < fixed['0.975quant'], 'non_sig','sig')
fixed$pc_increase = c(NA,(exp(fixed$mean[-1])-1)*100)
fixed

In [ ]:
for(i in seq(1,length(out.sre$marginals.fixed))){
    m = out.sre$marginals.fixed[[i]]
    print(paste0(round(inla.beta.marginal.summary(m)[[1]],2)," (",round(inla.beta.marginal.summary(m)[[2]],2),")"))
}

In [ ]:
write.table(fixed,paste0("../data/",filename,'_odds_sre.csv'), 
            col.names = TRUE, row.names = FALSE, sep=',')

### Predictive checks

In [ ]:
pp_checks[5,'model'] <- 'sre'
pp_checks[5,'ags_var'] <- '55'
pp_checks[5,'lik'] <- 'beta'
pp_checks[5,'bayes_r2_train'] <- bayes_pseudoR2(out.sre, data_train[['y_scaled']],index_obs = as.numeric(row.names(data_train)))
pp_checks[5,'bayes_r2_test'] <- bayes_pseudoR2(out.sre, data_test[['y_scaled']],index_obs = as.numeric(row.names(data_test)))
pp_checks[5,'waic'] <- out.sre$waic$waic
pp_checks[5,'cpo_fail'] <- sum(out.sre$cpo$failure, na.rm = TRUE)/dim(data)[1]*100
pp_checks

In [ ]:
pp_checks_train <- inla_ppcheck(out.sre, data_train[['y_scaled']], index_obs = as.numeric(row.names(data_train)))
pp_checks_test <- inla_ppcheck(out.sre, data_test[['y_scaled']], index_obs = as.numeric(row.names(data_test)))
options(repr.plot.width = 40, repr.plot.height = 12.5)
p <- ggplot_inla_ppcheck(pp_checks_train, pp_checks_test,CI = TRUE, binwidth=0.05)
ggsave(paste0("../plots/",filename,"_sre_pp_checks.pdf"), p, width = 42, height = 10)

plot(p)

## Save outputs

In [ ]:
## Plot the fixed parameters marginals for the coefficients

model_list <- list(out, out.sre)
model_name <- list('Beta-model','spatial Beta-model')
cf_list <- list()
for(m in seq(1,length(model_list))){ 
          
    rmf <- model_list[[m]]$marginals.fixed
    cf_list[[m]] <- data.frame(do.call(rbind, rmf))
    cf_list[[m]]$parameter <- rep(names(rmf),times=sapply(rmf,nrow))
    cf_list[[m]]$model <- as.character(model_name[[m]])
}
cf <- do.call(rbind, cf_list)
cf <- subset(cf, cf$parameter!='(Intercept)')

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 90)
p = ggplot(cf,aes(x=x,y=y,colour = model))+geom_line(size = 2.5,alpha=1)+
        facet_wrap(~ parameter, scales="free", nrow = 7)+geom_vline(xintercept=0, size = 2)+ylab("Density") + scale_color_manual(values=c("#1A85FF", "#D41159"))
invisible(cf)
p <- p + ggplot_theme + theme(strip.background = element_blank(), 
                              text = element_text(size=50),
                              strip.text.x = element_text(size = 50, face = "bold")) + xlab('') + theme(legend.position = "top") + theme(panel.spacing = unit(2, "lines")) + scale_x_continuous(labels = function(x) format(x, scientific = TRUE))
p
ggsave(paste0("../plots/",filename,"_fixed_sre.pdf"), width = 25, height = 90,limitsize = FALSE)

In [ ]:
## Plot the fixed parameters marginals for the odds ratio

model_list <- list(out, out.sre)
model_name <- list('Beta-model', 'spatial Beta-model')
cf_list <- list()
for(m in seq(1,length(model_list))){ 
          
    rmf <- model_list[[m]]$marginals.fixed
    odds <- list()
    for(c in seq(1, length(rmf))){
        odds[[c]] <- inla.tmarginal(function(z) (exp(z)-1)*100, rmf[[c]])
    }

    cf_list[[m]] <- data.frame(do.call(rbind, odds))
    cf_list[[m]]$parameter <- rep(names(rmf),times=sapply(odds,nrow))
    cf_list[[m]]$model <- as.character(model_name[[m]])
}
cf <- do.call(rbind, cf_list)
cf <- subset(cf, cf$parameter!='(Intercept)')

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 90)
p = ggplot(cf,aes(x=x,y=y,colour = model))+geom_line(size = 2.5,alpha=1)+
        facet_wrap(~ parameter, scales="free", nrow = 7)+geom_vline(xintercept=0, size = 2)+ylab("Density") + scale_color_manual(values=c("#1A85FF", "#D41159"))
invisible(cf)
p <- p + ggplot_theme + theme(strip.background = element_blank(), 
                              text = element_text(size=50),
                              strip.text.x = element_text(size = 50, face = "bold")) + xlab('') + theme(legend.position = "top") + theme(panel.spacing = unit(2, "lines"))
p
ggsave(paste0("../plots/",filename,"_fixed_sre_odds.pdf"), width = 25, height = 90,limitsize = FALSE)

In [ ]:
## Plot the hyperparameters marginals 
options(repr.plot.width = 55, repr.plot.height = 15)
p <- bri.hyperpar.plot(out.sre, together=FALSE)
p
ggsave(paste0("../plots/",filename,"_hyper_sre.pdf"), width = 30, height = 10)

In [ ]:
## Save predictive checks
write.table(pp_checks,paste0("../data/",filename,'_pp_checks.csv'), 
            col.names = TRUE, row.names = FALSE, sep=',')

In [ ]:
## Save posterior mean and sd for the predicted data (sre model)
data[,paste0(response_var, '_mean')] <- as.numeric(out.sre$summary.fitted.values[, "mean"])
data[,paste0(response_var, '_sd')] <- as.numeric(out.sre$summary.fitted.values[, "sd"])
write.table(data,paste0("../data/",filename,'_sre_fitted_values.csv'), 
            col.names = TRUE, row.names = FALSE, sep=',')

## Save posterior mean and sd for the random effects (sre model)
random = out.sre$summary.random$ID[,c('ID','mean','sd')]
write.table(random, paste0("../data/",filename,"_sre_random.csv"), sep = ',', row.names = F, col.names = T)

# Prior sensitivity checks

## 3. IID + Spatial random effect model - Beta likelihood, 55% retained AGS variance

### Priors 

In [ ]:
prior.iid.shape = c(0.001,0.1,1)
prior.iid.rate = c(0.0001,0.001,0.01,0.1)
prior.besag.shape = c(0.001,0.1,1)
prior.besag.rate = c(0.0001,0.001,0.01,0.1,0.0001)
prior.fixed.mean = c(0)
prior.fixed.prec = c(1)
priors <- expand.grid(prior.iid.shape,prior.iid.rate,
                      prior.besag.shape,prior.besag.rate,
                      prior.fixed.mean,prior.fixed.prec)
colnames(priors) <- c('prior.iid.shape','prior.iid.rate',
                      'prior.besag.shape','prior.besag.rate',
                      'prior.fixed.mean','prior.fixed.prec')
priors

In [ ]:
row <- 1
cf_list <- list()
while(row <= dim(priors)[1]){ 

    prior.iid = c(priors[row,'prior.iid.shape'],priors[row,'prior.iid.rate'])
    prior.besag = c(priors[row,'prior.besag.shape'],priors[row,'prior.besag.rate'])

    ff.sre <- as.formula(paste('y_scaled', paste(c("f(ID, model = 'bym', param = c(prior.iid,prior.besag), graph=data_adj, adjust.for.con.comp = TRUE, scale.model = TRUE)",
                                                    cov_55),
                                                    collapse=" + "),sep='~'))

    out.sre <- inla(ff.sre,data = data, 
                    family = 'beta',
                    control.predictor = list(link = 1), 
                    control.fixed = list(mean = priors[row,'prior.fixed.mean'], 
                                            prec = priors[row,'prior.fixed.prec'], 
                                            prec.intercept=1),
                    num.threads=1,
                    verbose = TRUE)
    out.sre = inla.rerun(out.sre)
    summary(out.sre)

    rmf = out.sre$marginals.fixed
    cf_list[[row]] = data.frame(do.call(rbind, rmf))
    cf_list[[row]]$parameter = rep(names(rmf),times=sapply(rmf,nrow))
    cf_list[[row]]$row <- as.character(row)

    row <- row + 1
}
cf <- do.call(rbind, cf_list)


In [ ]:
options(repr.plot.width = 55, repr.plot.height = 20)
p = ggplot(cf,aes(x=x,y=y,colour = row))+geom_line(size = 0.5,alpha=0.3)+
        facet_wrap(~ parameter, scales="free", nrow=2)+geom_vline(xintercept=0)+ylab("Density")
invisible(cf)
p <- p + ggplot_theme + theme(strip.background = element_blank(), 
                                strip.text.x = element_text(size = 50, face = "bold")) + xlab('') + theme(legend.position = "none") +scale_color_manual(values=rep('blue', row-1)) + theme(panel.spacing = unit(2, "lines"))
p
ggsave(paste0("../plots/",filename,"_prior_sens_sre.pdf"), width = 55, height = 20, limitsize = FALSE)


In [ ]:
inla.show.hyperspec(out.sre)